In [1]:
%matplotlib widget

In [2]:
import numpy as np
import scipy as sp

from pyinverse.axes import RegularAxes3
from pyinverse.grid import RegularGrid
from pyinverse.ray3 import regular_axes2polytope, grid_uv2half_planes, beam2actor
from pyviz3d.viz import Renderer

In [3]:
N1 = 5
N2 = 8
N3 = 6

axes3 = RegularAxes3.linspace((-1, 1, N1), (-1, 1, N2), (-1, 1, N3))

Nu = 7
Nv = 8

uv_grid = RegularGrid.linspace((-1, 1, Nu), (-1, 1, Nv))

In [4]:
ijk = (1, 0, 0)

In [5]:
voxel_ijk = axes3.voxel_actor(ijk, color='red')

In [6]:
N_hires = 65
# N_hires = 127

axes3_voxel_hires = RegularAxes3.linspace((-1, 1, N_hires), (-1, 1, N_hires), (-1, 1, N_hires))
axes3_beam_hires = RegularAxes3.linspace((-1, 1, N_hires), (-1, 1, N_hires), (-1, 1, N_hires))

In [7]:
ijk = (1, 0, 0)
A_ijk, b_ijk = regular_axes2polytope(axes3, ijk)

In [8]:
P_hires = np.array((axes3_voxel_hires.centers))

In [9]:
AP_hires = np.einsum('ij,jklm->iklm', np.array(A_ijk), P_hires)
AP_hires.shape = 6, N_hires**3

In [10]:
I = [(x <= b_ijk).all() for x in AP_hires.T]
I_blank = np.where(~np.array(I))

In [11]:
X_hires = np.zeros((N_hires, N_hires, N_hires))
I_indicator = np.where(np.array(I))
X_hires.flat[I_indicator] = 1
X_hires.flat[I_blank] = np.nan

In [12]:
X_ijk_actor = axes3_voxel_hires.actor(X_hires, vmin=0, vmax=1, blank_nan=True)

In [13]:
theta = np.pi/8
phi = np.pi/3
mn = (1, 1)

A_mn, b_mn = grid_uv2half_planes(theta, phi, uv_grid, mn)

beam_mn_actor = beam2actor(uv_grid, mn, (-1.5*np.sqrt(2), 1.5*np.sqrt(2)), theta, phi)

In [14]:
AuvP_hires = np.einsum('ij,jklm->iklm', A_mn, P_hires)
AuvP_hires.shape = 4, N_hires**3

In [15]:
Iuv = [(x <= b_mn).all() for x in AuvP_hires.T]
Iuv_blank = np.where(~np.array(Iuv))

In [16]:
Xuv_hires = np.zeros((N_hires, N_hires, N_hires))
Iuv_indicator = np.where(np.array(Iuv))
Xuv_hires.flat[Iuv_indicator] = 1
Xuv_hires.flat[Iuv_blank] = np.nan

In [17]:
Xuv_mn_actor = axes3_beam_hires.actor(Xuv_hires, vmin=0, vmax=1, blank_nan=True)

In [18]:
ren = Renderer()
ren.add_actor(voxel_ijk)
ren.add_actor(X_ijk_actor)
ren.add_actor(beam_mn_actor)
ren.add_actor(Xuv_mn_actor)
ren.axes_on((-1, 1, -1, 1, -1, 1))
ren.reset_camera()
ren.start()